In [1]:
!pip install pygsp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 4.3 MB/s 


In [2]:
from sklearn.covariance import GraphicalLasso as glasso
import numpy as np
from scipy.io import loadmat
import numpy as np 
import matplotlib.pyplot as plt 
from time import time 
import networkx as nx
import pygsp
from sklearn import preprocessing
from scipy.optimize import linprog
from scipy.spatial.distance import pdist
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigs
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import normalized_mutual_info_score as nmi
from scipy.optimize import bisect
from scipy import sparse
from scipy.special import expit
from scipy.spatial.distance import cosine
#import cvxpy as cp
#from cvxpy.atoms.elementwise import logistic
from sympy import *
import pickle
import timeit
def save_variable(variable,filename):
  pickle.dump(variable,open(filename, "wb"))
def load_variable(filename):
  return pickle.load(open(filename,'rb')) 
def NormalizeData(data):
  X_std = (data - np.min(data)) / (np.max(data) - np.min(data))
  X_scaled = X_std * (1 - 0) + 1
  return X_scaled

In [3]:
def UpdateW(X,Q,c,beta1,beta2,max_iter,rho_w,w_init):
  w = w_init
  N = len(X[:,0])
  D = len(X[0,:])
  c.shape = (N,1)
  w.shape = (D,2)
  B1 = X.T@np.append(c,np.ones((N,1)),1)-beta2*w
  for j in range(max_iter):
    Q = expit(np.append(c,np.ones((N,1)),axis=1)@w.T)
    w = w + rho_w*(B1-Q.T@np.append(c,np.ones((N,1)),1))  
    if (sum(sum((B1-Q.T@np.append(c,np.ones((N,1)),1))<1.0e-4*np.ones((len(X[0,:]),2))))==2*len(X[0,:])):
      print('w Converged')
      break
  return Q,w

def UpdateC(X,Q,w,abstheta,rho_c,c_init,max_iter,m,beta1):
  c = c_init
  N = len(X[:,0])
  D = len(X[0,:])
  obj = []
  c.shape = (N,1)
  w_ = w[:,0]
  w_.shape = (D,1)
  A1 = (abstheta@np.ones((N,1)))+X@w_
  for j in range(max_iter):
    Q = expit(np.append(c,np.ones((N,1)),axis=1)@w.T)
    c = c + rho_c*(A1-Q@w_)

    def f(x):
      return sum([min(max(c[k]-x,0),1) for k in range(N)])-m   
    lamda =  bisect(f,-10000,10000) 

    for i in        range(N):
      c[i] = min(max(c[i]-lamda,0),1) 
    obj.append(sum(abs(A0)@c)+beta1*(sum(sum(X*np.log(Q+0.001)))+sum(sum((np.ones(np.shape(X))-X)*np.log(np.ones(np.shape(X))-Q+0.001)))))
    if j>1:
      if (abs(obj[j]-obj[j-1])<1.0e-4): 
        print('c Converged')
        break  
  return Q,c

In [ ]:
# Loading data:

# Facebook
f = open("/content/drive/MyDrive/datasets/journal/fb/0.edges")
Lines = f.readlines()
edges = []
for line in Lines:
   edges.append(line.split(" "))
f.close()
A_=np.zeros(np.shape(edges))
for i in range(len(edges)):
  for j in range(len(edges[0])):
    A_[i,j] = int(edges[i][j])-1
A0_ = A_ 

f = open("/content/drive/MyDrive/datasets/journal/fb/0.feat")
Lines = f.readlines()
X_ = []
for line in Lines:
  X_.append(line.split(" "))
f.close()
X__=np.zeros(np.shape(X_))
for i in range(len(X_)):
  for j in range(len(X_[0])):
    X__[i,j] = float(X_[i][j])

X = X__[:,1:len(X__[0,:])]
N=len(X[:,0])
A0 = sparse.csr_matrix((np.ones(len(A0_[:,0])), (A0_[:,0], A0_[:,1])), shape=(N, N)).toarray()

'''
To load the other datasets in the paper:

# Twitter
data=loadmat('/Datasets/twitter/twitter.mat')
A0 = np.array(data['A0'].todense())
X = data['X']
N = len(A0[:,0])


# Gplus
data=loadmat('/Datasets/gplus/gplus.mat')
A0 = np.array(data['A0'].todense())
X = data['X']
N = len(A0[:,0])


# Cora
data=loadmat('/Datasets/cora.mat')
A0 = data['A0']
X = data['X']
N = len(A0[:,0])


# Synthetic data
A0 = load_variable('/Datasets/syn/theta_syn')
N = np.size(A0,0)
X = load_variable('/Datasets/syn/ga_affine_bool')
D = np.size(X,1)  
c_true = load_variable('/Datasets/syn/c_syn')   
'''

In [ ]:
beta1 = 1
beta2 = 1
max_iter = 1000
obj_fn = []
w = 0.1*np.ones((len(X[0,:]),2))
c=np.ones((N,))
Q = np.zeros(np.shape(X))
for m in range (0,max_iter):   
  #Update c---------------------------------------------------------------------
    Q,c = UpdateC(X,Q,w,abs(A0),1e-4*np.ones((N,1)),c,
                10**3,m=N/4,beta1=1) 
    print("c=",c.T)

  #Update w---------------------------------------------------------------------
    Q,w = UpdateW(X,Q,c,beta1,beta2,10**3,1e-4*np.ones((len(X[0,:]),1)),w) #X,Q,c,beta1,beta2,,max_iter,rho_w,w_init
    
    obj_fn.append(sum(abs(A0)@c)+(sum(sum(X*np.log(Q+1e-100)))+sum(sum((np.ones(np.shape(X))-X)*np.log(np.ones(np.shape(X))-Q+1e-100))))-beta2*np.linalg.norm(w)**2)
############--------------------------------------------------------------------
    if (m>0):
      if (abs(obj_fn[m]-obj_fn[m-1])<10**(-4)):
        break